# 1. Install and Import Dependencies

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install transformers requests beautifulsoup4 pandas numpy
!pip install simpletransformers requests beautifulsoup4 pandas numpy
!pip install sentencepiece
!pip install nltk
!pip install seaborn
!pip install matplotlib
import pandas as pd
import nltk
import seaborn as sea
import matplotlib as plt
import numpy as np
import xgboost as xgb
import pickle
import regex as re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, roc_curve, auc, recall_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.1 MB/s eta 0:00:00
    

# 2. Instantiate LLM Model

In [2]:
import sentencepiece
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# Install sentencepiece if not already installed
# You can run this in your terminal or command prompt
# pip install sentencepiece

# Load the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('akhooli/xlm-r-large-arabic-sent')

# Load the model
model = XLMRobertaForSequenceClassification.from_pretrained('akhooli/xlm-r-large-arabic-sent')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identica

# 3. Encode and Calculate Sentiment

**Inputs**

In [38]:
description = ''
rating = '5'



**LLM Model Sentiment Analysis**

In [39]:
tokens = tokenizer.encode(description, return_tensors='pt')
result = model(tokens)
sentiment_score=int(torch.argmax(result.logits))+1
sentiment_score

2

**Decode LLM Model Results**

In [40]:
def decode_sentiment(sentiment_score):
    if sentiment_score == 1:
        return 'neutral'
    elif sentiment_score == 2:
        return 'negative'
    elif sentiment_score == 3:
        return 'positive'
    else:
        return 'unknown'

# Example usage:
#score = 2
LLM = decode_sentiment(sentiment_score)
print(f"Sentiment: {LLM}")




Sentiment: negative


**Builting Features**

In [41]:
import pandas as pd
"""data = pd.read_excel('/content/Jan_24_Rating.xlsx')
data.head()"""
data = pd.DataFrame({
    'LLM': [LLM],
    'description': [description],
    'rating': [rating]
})

In [42]:
#data.head()

In [43]:
import pandas as pd
import regex as re
import numpy as np

# Assuming you have a DataFrame named 'data'

Positive_Strings = ['Fantastic product','Thx','Love the product','Love', 'love','LOVE', 'applause','exceptional','Alhumdulillah','Allhamdulliah', 'Alhamdulillah', 'Masha Allah','mashallah', 'Mashallah', 'MaShaAllah', 'Allah', 'Thank', 'thank', 'Thanks', 'Thank you', 'thank you',
                    'Behtareen','Hats off','Behtereen', 'behtareen', 'awesome', 'Awesome', 'VIP', 'vip', 'Vip', 'Best', 'best', 'good product','Good packing', 'nyc', 'nice', 'Nice', 'Allaw', 'allaw','',
                    'orginal product', 'thnx', 'Acha product', 'Best', 'ZabbarDast', 'Zabrdast', 'zabrdast', 'Zbrdast', 'Satisfactory', 'satisfactory', 'Bht zada acha','achi quality', 'Satisfied with Product',
                    'satisfying','Not bad','shukiya','acchi hai','Outstanding','keep it up','Aala product','geniue','safe','Wonderful','Great service','thank', 'Great quality', 'Good quality', 'good quality',
                    'Bhut he Aala', 'happy', 'Happy', '#Trusted', '#trusted', 'Trusted', 'perfect', 'Perfect', 'Excellent', 'appreciated', 'Highly recommend', 'Amazing', 'glad', 'Outstanding', 'Bhot khoob','Good service', 'Totally satisfied',
                    'Nice product','decent','value for money', 'value of money','Original Product','behtreeen','5 stars','Original product','Bohat ache','Unbeatable prices','10/10','decent','great product','Impressive','Very good service','good service','100% satisfied',
                    'Zbardast','Zabardast','Good e-commerce web site','Good web site','good web site','Good experience','good experience', 'Genuine', 'pretty good', 'high quality', 'Product original', 'Product is original', 'Shandar product', 'better quality', 'Good overall', 'Genuine product', '100% original', 'Fantastic', 'fantastic']


Negative_Strings = ['Not much satisfied','not too good', 'kam h','kharb','nahi','Bad','Bakwassssssss', 'bad', 'Ghatya', 'ghatya', 'Fruad', 'fraud', 'bakwas', 'Bakwas', 'poor', 'Poor', 'missing', 'cheap', 'Cheap', 'not satisfied', 'not recommended', '3rd class',
                    'Not recommended', 'not good', 'local quality','waste of money', 'scratches', 'scratch', 'pathetic','ghatiya','Broken', 'damaged', 'repacked','Repacked','Defective','defective',
                    'Low quality', 'Baqwas','poor', 'faulty product', 'issue','Not satisfied.', "didn't work", 'kharab', 'Kharab', 'Pathetic', 'too late', 'late', 'Late', 'poor', 'unsatisfactory' ]

conditions_accepted_1 = (data['description'].astype(str).str.contains('|'.join(Positive_Strings))&(data['rating'].astype(str).str.contains('3|4|5')))
conditions_accepted_2 = ((data['rating'].astype(str).str.contains('4|5')) & (data['description'].isna()))

# Conditions
conditions_accepted_3 = (
((data['rating'].astype(str).str.contains('4|5'))&
 (data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False))&
 (data['LLM'].str.contains('positive|Positive', case = False))) |

((data['LLM'].str.contains('Positive|positive', case=False)) &
 (data['rating'].astype(str).str.contains('3')) &
 (data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False)))|

((data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False))&
(data['LLM'].str.contains('neutral|Neutral', case=False)) &
(data['rating'].astype(str).str.contains('4|5') )) |

((data['LLM'].str.contains('Neutral|neutral', case=False)) &
(data['rating'].astype(str).str.contains('3')) &
(data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False))) |

((data['LLM'].str.contains('negative|Negative', case=False)) &
(data['rating'].astype(str).str.contains('4|5')) &
(data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False))) |

((data['LLM'].str.contains('negative|Negative', case=False)) &
(data['rating'].astype(str).str.contains('3')) &
(data['description'].astype(str).str.contains('|'.join(Positive_Strings), case=False)))

                          )

# Now For Rejected
conditions_rejected_1 =  (data['rating'].astype(str).str.contains('1|2'))
conditions_rejected_2 = (data['description'].astype(str).str.contains('|'.join(Negative_Strings))&(data['rating'].astype(str).str.contains('1|2|3|4|5')))
conditions_rejected_3 = (
(data['LLM'].str.contains('Positive|positive', case=False) &
(data['rating'].astype(str).str.contains('3|4|5'))&
(data['description'].astype(str).str.contains('|'.join(Negative_Strings), case=False))) |

(data['LLM'].str.contains('neutral|Neutral', case=False) &
(data['rating'].astype(str).str.contains('3|4|5'))&
(data['description'].astype(str).str.contains('|'.join(Negative_Strings), case=False))) |

(data['LLM'].str.contains('negative|Negative', case=False) &
(data['rating'].astype(str).str.contains('3|4|5'))&
(data['description'].astype(str).str.contains('|'.join(Negative_Strings), case=False)))

                       )







# Display the resulting DataFrame
#data.to_excel('Labled_Data.xlsx', index =False)



**Prediction on the basis of LLM And Features**

**Mati Bhai accepted k case mein, Hum Onclick function apply kray gy, on "accepted" Button. Baki jo Senerio hai, Us mein vese hee rehnain dena hai, QK 80% ACCEPT K  hee cases hotay hai**

In [44]:
# Applying labels based on conditions
"""data['Labeled Result'] = np.select([conditions_accepted, conditions_rejected, conditions_ignore],
                                   ['accepted', 'rejected', 'ignore'],
                                   default='ignore')"""

# Update the conditions to match the length of the DataFrame index
data['Labeled Result'] = np.select(
    [(conditions_accepted_1 | conditions_accepted_2 | conditions_accepted_3), (conditions_rejected_1 | conditions_rejected_2 |conditions_rejected_3)],
    ['accepted', 'rejected'],
    default='rejected'
)

data

# Display the resulting DataFrame
#data.to_excel('Labled_Data.xlsx', index =False)
#data
#data.to_excel('Data_Verifying_last.xlsx', index =False)

,LLM,description,rating,Labeled Result
0,negative,,5,accepted


**Uses Trained Model For Prediction**

In [ ]:
"""
data['Feature_1'] = conditions_accepted
data['Feature_2'] = conditions_rejected
data['Feature_3'] = conditions_ignore
#data

data_to_model = data[['Feature_1', 'Feature_2', 'Feature_3']]
data_to_model

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the trained SVM model
model_filename = '/content/Review_Rating_Model_SVM.pkl'
with open(model_filename, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Use label encoding to convert textual labels to numerical values for the 'Feature' columns
label_encoder = LabelEncoder()

# Use .loc to set values in the original DataFrame
data_to_model.loc[:, 'Feature_1'] = label_encoder.fit_transform(data['Feature_1'])
data_to_model.loc[:, 'Feature_2'] = label_encoder.fit_transform(data['Feature_2'])
data_to_model.loc[:, 'Feature_3'] = label_encoder.fit_transform(data['Feature_3'])

def decode_prediction(y_pred_encoded):
    if y_pred_encoded == 0:
        return 'accepted'
    elif y_pred_encoded == 1:
        return 'rejected'
    elif y_pred_encoded == 2:
        return 'ignore'
    else:
        return 'unknown'

# Make predictions using the loaded model
y_pred_encoded = loaded_model.predict(data_to_model)
print(decode_prediction(y_pred_encoded))

"""

"\ndata['Feature_1'] = conditions_accepted\ndata['Feature_2'] = conditions_rejected\ndata['Feature_3'] = conditions_ignore\n#data\n\ndata_to_model = data[['Feature_1', 'Feature_2', 'Feature_3']]\ndata_to_model\n\nimport pickle\nimport pandas as pd\nfrom sklearn.preprocessing import LabelEncoder\n\n# Load the trained SVM model\nmodel_filename = '/content/Review_Rating_Model_SVM.pkl'\nwith open(model_filename, 'rb') as model_file:\n    loaded_model = pickle.load(model_file)\n\n# Use label encoding to convert textual labels to numerical values for the 'Feature' columns\nlabel_encoder = LabelEncoder()\n\n# Use .loc to set values in the original DataFrame\ndata_to_model.loc[:, 'Feature_1'] = label_encoder.fit_transform(data['Feature_1'])\ndata_to_model.loc[:, 'Feature_2'] = label_encoder.fit_transform(data['Feature_2'])\ndata_to_model.loc[:, 'Feature_3'] = label_encoder.fit_transform(data['Feature_3'])\n\ndef decode_prediction(y_pred_encoded):\n    if y_pred_encoded == 0:\n        return 'a